<a href="https://colab.research.google.com/github/sio-co2o2/keelingcurve_notebooks/blob/main/notebooks/create_graphic_full_mlo_record_keelingcurve.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/sio-co2o2/keelingcurve_notebooks/main?labpath=notebooks/create_graphic_full_mlo_record_keelingcurve.ipynb)

# How to make changes to a graphic appearing on the Keeling Curve website
### **Graphic: CO<sub>2</sub> full record at Mauna Loa, Hawaii**

#### This notebook illustrates how to create a customized PDF and PNG images of a graphic appearing on the front page of [keelingcurve.ucsd.edu](keelingcurve.ucsd.edu).

For an overview of the Keeling Curve Jupyter notebook, see [notebooks overview](https://keelingcurve.ucsd.edu/overview-of-keeling-curve-jupyter-notebooks/https://keelingcurve.ucsd.edu/overview-of-keeling-curve-jupyter-notebooks/).

**To use the notebook, first run all the cells.** 

In Google Colab, select from the top menu and select Runtime -> Run all. Then buttons are activated in the 'creating the graphics setction' to click on to run the code to create and download the graphic.

Direct questions to [webmaster-co2o2-sio@ucsd.edu](mailto:webmaster-co2o2-sio@ucsd.edu)

---

### [Go to creating the graphic section](#button-to-create-and-save-the-graphic)

---

### Sample of the graphic produced by this notebook

![title](https://github.com/sio-co2o2/keelingcurve_notebooks/blob/main/images/sample_graphics/mlo_full_record.png?raw=1?modified=12345678)

## The following python code creates the graphic and the code can be modified to create changes to the graphic

## Section: How to setup Google Colab to run the Jupyter code

### First, any external code needed by this notebook has to be imported

When the notebook is run using Google Colab, the import statements install requested packages into the Colab python environment.

### Import python packages

In [2]:
import os
import functools
import pathlib

import pandas as pd
import numpy as np

from datetime import datetime, date, timedelta

import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator
from matplotlib import ticker
import matplotlib.colors as mcolors
from matplotlib.patches import Polygon

from ipywidgets import widgets
from IPython.display import HTML, display

from PIL import Image

%matplotlib inline

### Import the following package differently than above

The package cairosvg used to include the SIO vector svg logo in the graphic is not a standard package available to Google Colab, so it must be installed using pip. 

In [3]:
# This package is used to convert a vector svg into a png
try:
  from cairosvg import svg2png
except:
  ! pip install cairosvg
  from cairosvg import svg2png

In [4]:
# This import is to enable Google Colab to save files and then download them

# This import does not exist unless the notebook is run in Google Colab
# Put in a try except block if user wants to use notebook off of Google Colab

try:
    from google.colab import files
except:
    pass

## Section: How to set locations for data and logo files

### Setup the directories and file names globally

Then this setup can be accessed from any function in the notebook

In [5]:
# Read in data from github repository
# Get mlo record to last daily value
data_file = 'https://raw.githubusercontent.com/sio-co2o2/keelingcurve_notebooks/main/preliminary_data/mlo/mlo_full_record_now_span.csv'

logo_file = 'https://github.com/sio-co2o2/keelingcurve_notebooks/raw/main/images/logos/ucsd_sio_logo.svg'

plot_dir = pathlib.Path('./plots')
plot_dir.mkdir(exist_ok=True)

plot_basename = 'mlo_full_record_example'

pdf_file = plot_dir / f'{plot_basename}.pdf'
png_file = plot_dir / f'{plot_basename}.png'

## Section: How to load in data and prepare it for plotting

### Load in the MLO record data using pandas

Because the data is in a csv format, one can use pandas to read it in and skip comments and not include any initial spaces in the data columns. Pandas will read the data into a table of rows and columns called a dataframe.

In [6]:
df = pd.read_csv(data_file,skipinitialspace=True,comment='"')
df.head()

,date,co2,date_seas_adj,co2_seas_adj,date_fit,co2_fit,date_seas_adj_fit,co2_seas_adj_fit,date_filled,co2_filled,date_seas_adj_filled,co2_seas_adj_filled
0,1958.041096,NaN,1958.041096,NaN,1958.041096,NaN,1958.041096,NaN,1958.041096,NaN,1958.041096,NaN
1,1958.126027,NaN,1958.126027,NaN,1958.126027,NaN,1958.126027,NaN,1958.126027,NaN,1958.126027,NaN
2,1958.202740,315.71,1958.202740,314.44,1958.202740,316.19,1958.202740,314.90,1958.202740,315.71,1958.202740,314.44
3,1958.287671,317.45,1958.287671,315.16,1958.287671,317.29,1958.287671,314.98,1958.287671,317.45,1958.287671,315.16
4,1958.369863,317.51,1958.369863,314.69,1958.369863,317.88,1958.369863,315.06,1958.369863,317.51,1958.369863,314.69


### Prepare the data for plotting

### Break this dataframe up into a data table and a fit table

This is in order to remove rows with null values in a set of columns without affecting other sets of columns.

In [7]:
df_co2 = df[['date', 'co2']].copy()
df_co2_fit = df[['date_fit', 'co2_fit']].copy()

### Remove rows with fill values -99.99 or NaN

In [8]:
df_co2 = df_co2[df_co2['co2'] != -99.99]
df_co2_fit = df_co2_fit[df_co2_fit['co2_fit'] != -99.99]

df_co2 = df_co2.dropna()
df_co2_fit = df_co2_fit.dropna()

df_co2.tail()

,date,co2
794,2023.480822,422.55
795,2023.483562,421.39
796,2023.486301,421.24
797,2023.491781,422.85
798,2023.494521,422.88


## Extract out Maunakea (MKO) data

In the MLO record, MKO data is included in the gap when the MLO record was stopped due to a volcanic eruption. To differentiate MKO data later when plotting, extract it out from each table.

#### Get start and end decimal dates of MKO data (12/1/2022 to 3/8/2023)

Monthly data is at 24:00 hours of the 15th of each month

Which means MKO has monthly data points from 12/16/2022 to 2/16/2023

But also want to set limits on any weekly and daily points used at the end of the plot,
so use the dates 12/1/2022 - 3/8/2023 since it also captures the monthly points

In [9]:
mauna_kea_start_date = datetime(2022, 12, 1)

year = mauna_kea_start_date.year
boy = datetime(year, 1, 1)
eoy = datetime(year + 1, 1, 1)
mko_decimal_start_date = year + ((mauna_kea_start_date - boy).total_seconds() / ((eoy - boy).total_seconds()))

mauna_kea_end_date = datetime(2023, 3, 8)

year = mauna_kea_end_date.year
boy = datetime(year, 1, 1)
eoy = datetime(year + 1, 1, 1)
mko_decimal_end_date = year + ((mauna_kea_end_date - boy).total_seconds() / ((eoy - boy).total_seconds()))


#### Filter data to get MKO data

In [10]:
df_co2_mko = df_co2[(df_co2['date'] >= mko_decimal_start_date) & (df_co2['date'] <= mko_decimal_end_date)]

df_co2_fit_mko = df_co2_fit[(df_co2_fit['date_fit'] >= mko_decimal_start_date) & (df_co2_fit['date_fit'] <= mko_decimal_end_date)]

### Use Numpy to save the tables as arrays to plot with later

In [11]:
date_co2 = df_co2['date'].to_numpy()
co2 = df_co2['co2'].to_numpy()

date_co2_fit = df_co2_fit['date_fit'].to_numpy()
co2_fit = df_co2_fit['co2_fit'].to_numpy()


date_co2_mko = df_co2_mko['date'].to_numpy()
co2_mko = df_co2_mko['co2'].to_numpy()

date_co2_fit_mko = df_co2_fit_mko['date_fit'].to_numpy()
co2_fit_mko = df_co2_fit_mko['co2_fit'].to_numpy()

## **Section:** Functions that will perform accessory operations but not change the graphic

These are functions need to convert dates that will not be modified when changing code used for plotting.

Used for plot limits and labeling

### Function to convert a timestamp in a python datetime format into a float format

In [12]:
#  https://stackoverflow.com/questions/19305991/convert-fractional-years-to-a-real-date-in-python
def dt2t(adatetime):
    """
    Convert adatetime into a float. The integer part of the float should
    represent the year.
    Order should be preserved. If adate<bdate, then d2t(adate)<d2t(bdate)
    time distances should be preserved: If bdate-adate=ddate-cdate then
    dt2t(bdate)-dt2t(adate) = dt2t(ddate)-dt2t(cdate)
    """
    year = adatetime.year
    boy = datetime(year, 1, 1)
    eoy = datetime(year + 1, 1, 1)
    return year + ((adatetime - boy).total_seconds() / ((eoy - boy).total_seconds()))

### Function to convert a timestamp in a float format to a datetime format

In [13]:
#  https://stackoverflow.com/questions/19305991/convert-fractional-years-to-a-real-date-in-python
def t2dt(atime):
    """
    Convert atime (a float) to DT.datetime
    This is the inverse of dt2t.
    assert dt2t(t2dt(atime)) == atime
    """
    year = int(atime)
    remainder = atime - year
    boy = datetime(year, 1, 1)
    eoy = datetime(year + 1, 1, 1)
    seconds = remainder * (eoy - boy).total_seconds()
    return boy + timedelta(seconds=seconds)

### Function to create various date versions for today's date

This is useful for graphics inset text and titles

In [14]:
def get_todays_date_variations():
    
    now = datetime.now()
    todays_decimal =  dt2t(now)
    
    today = date.today()

    todays_day = today.day
    todays_month = today.strftime("%B")
    todays_year = today.year
    todays_date_moyr = today.strftime("%B %Y")
    todays_date_modyyr = f"{todays_month} {todays_day}, {todays_year}"

    return todays_date_moyr, todays_date_modyyr, todays_decimal, todays_year

### Function to get the last date of the data

This can be used for graphic inset text and titles

In [15]:
def get_data_end_date(date_data):
    last_date = np.max(date_data)
    last_date_datetime = t2dt(last_date)
    
    # Convert date format to month_name day, year
    day = last_date_datetime.day
    month = last_date_datetime.strftime("%B")
    year = last_date_datetime.year
    date_modyyr = f"{month} {day}, {year}"
    return date_modyyr

## **Section:** Function to display the gradient under a curve in a graphic

The function gradient_fill modifies the gradient under a curve through changing the arguments passed to the function. The following can be specified: fill color of the gradient under the curve at full opacity, the plot limits of the gradient, the opacity of the gradient at the bottom of the graphic, and the layer order of adding the gradient to a plot.

### Function definition to create a gradient under a curve

This function has been modified from the original function, found on stackoverflow.com, to allow more flexibility in how the gradient is displayed.

This function doesn't need to be modified in order to change how the gradient is displayed.

In [16]:
# Gradient function modified from the original function in
# the answer from https://stackoverflow.com/questions/29321835/is-it-possible-to-get-color-gradients-under-curve-in-matplotlib

# Modified to add gradient below curve and have a bottom alpha
# The color under the plot line varies as a gradient from the line to the
# bottom axis.

# Modify arguments of the original function to include limits of the 
# gradient under the curve, the transparency and gradient color
# def gradient_fill(x, y, fill_color=None, ax=None, **kwargs):
def gradient_fill(x, y, fill_color='#FFFFFF', xmin=None, ymin=None, xmax=None, 
                  ymax=None, alpha_bottom=None, ax=None, alpha=1, zorder=1, **kwargs):
    """
    Plot a line with a linear alpha gradient filled beneath it.

    Parameters
    ----------
    x, y : array-like
        The data values of the line.
    fill_color : a matplotlib color specifier (string, tuple) or None
        The color for the fill. If None, the color of the line will be used.
    ax : a matplotlib Axes instance
        The axes to plot on. If None, the current pyplot axes will be used.
    Additional arguments are passed on to matplotlib's ``plot`` function.

    Returns
    -------
    line : a Line2D instance
        The line plotted.
    im : an AxesImage instance
        The transparent gradient clipped to just the area beneath the curve.
    """
    if ax is None:
        ax = plt.gca()

    # Only want the gradient to show and not the line defining the gradient shape
    # so change the original function
    #line, = ax.plot(x, y, **kwargs)
    #if fill_color is None:
    #    fill_color = line.get_color()

    if alpha_bottom is None:
        alpha_bottom = 0

    if xmin is None:
        xmin = x.min()

    if ymin is None:
        ymin = y.min()
        
    if xmax is None:
        xmax = x.max()

    if ymax is None:
        ymax = y.max()
        
    if zorder is None:
        zorder = 1
        
    # zorder in alpha of the gradient are now set in the function arguments,
    # so modify this portion of the original function
    #zorder = line.get_zorder()
    
    #alpha = line.get_alpha()
    #alpha = 1.0 if alpha is None else alpha

    z = np.empty((100, 1, 4), dtype=float)

    rgb = mcolors.colorConverter.to_rgb(fill_color)
    z[:, :, :3] = rgb
    #z[:,:,-1] = np.linspace(0, alpha, 100)[:,None]
    z[:, :, -1] = np.linspace(alpha_bottom, alpha, 100)[:, None]

    # Get the gradient limits from the function arguments,
    # so modify the original function
    #xmin, xmax, ymin, ymax = x.min(), x.max(), y.min(), y.max()
    
    im = ax.imshow(z, aspect='auto', extent=[xmin, xmax, ymin, ymax],
                   origin='lower', zorder=zorder)

    xy = np.column_stack([x, y])
    xy = np.vstack([[xmin, ymin], xy, [xmax, ymin], [xmin, ymin]])
    clip_path = Polygon(xy, facecolor='none', edgecolor='none', closed=True)
    ax.add_patch(clip_path)
    im.set_clip_path(clip_path)
    
    ax.autoscale(True)
    
    # Modify original function since there is no need 
    # to return the line and im objects
    #return line, im

<a name="functions-to-modify-graphic"></a>
## **Section**: Functions that can be modified to change and display the graphic

Examples of modifications are modifying fonts, line-widths, plot symbols, axes, and plot range.

### Function to set global fonts and linewidth properties

In the function 'set_matplotlib_properties', the font properties used for titles and labels of the graphic are set. The default axes linewidth is also set here.


In [17]:
def set_matplotlib_properties():
    
    # Set properties for matplotlib
    
    # Reset rcparams in case modified defaults in another notebook during same session 
    plt.rcParams.update(plt.rcParamsDefault)

    plt.rcParams.update({'axes.linewidth':1.5})
    
    plt.rcParams.update({
        "text.usetex": False,
        "font.family": "sans-serif",
        "font.weight":  "normal",
        "font.sans-serif": ["Arial", "Tahoma", "Helvetica","FreeSans", "NimbusSans", "LiberationSans","DejaVu Sans"],
        "mathtext.default":'regular',
        "mathtext.fontset": "dejavusans"
    })
        
    # http://phyletica.org/matplotlib-fonts/
    # This tells matplotlib to use Type 42 (a.k.a. TrueType) fonts 
    # for PostScript and PDF files. This allows you to avoid Type 3 fonts.
    # Turning on usetex also works
    
    # This is needed on Ubuntu for pdf fonts 
    plt.rcParams.update({
      'pdf.fonttype': 42,
        'ps.fonttype': 42 
    })

### Function to set global properties of the plot

The function 'set_plot_props' declares axes tick properties, x and y axis label properties, and x & y axes limits for the plot 

In [18]:
def set_plot_props(ax, fig, xmin, xmax, ymin, ymax, xlabel, ylabel):

    # ---------------------------------
    # Plot properties for website plots
    # ---------------------------------

    # Allow room at top for the 2 titles
    fig.subplots_adjust(top=0.85)

    ax.tick_params(which='both', bottom=True, top=True, left=True, right=True)

    ax.tick_params(axis='x', labelsize=12)
    ax.tick_params(axis='y', labelsize=16)

    ax.tick_params(axis='x', which='major', direction='in', length=8, width=1)
    ax.tick_params(axis='y', which='major', direction='in', length=8, width=1)

    tick_spacing = 5
    ax.xaxis.set_major_locator(ticker.MultipleLocator(tick_spacing))
    
    tick_spacing = 5
    ax.yaxis.set_major_locator(ticker.MultipleLocator(tick_spacing))
    
    labels = ax.get_xticklabels() + ax.get_yticklabels()
    for label in labels: label.set_fontweight('bold')   
    
    # Display every other y major tick label
    for label in ax.yaxis.get_ticklabels()[::2]:
        label.set_visible(False)

    tick_length = 5
    ax.tick_params(axis='x',which='minor', direction='in', length=tick_length)
    
    tick_length = 4
    ax.tick_params(axis='y',which='minor', direction='in', length=tick_length)
    
    ax.xaxis.set_minor_locator(AutoMinorLocator(5))
    ax.yaxis.set_minor_locator(AutoMinorLocator(5))
      
    ax.set_xlabel(xlabel, fontweight='bold', fontsize=18, labelpad=5)
    ax.set_ylabel(ylabel, fontweight='bold', fontsize=18, labelpad=5)

    ax.set_xlim(xmin, xmax)
    ax.set_ylim(ymin, ymax)

### Function to place the titles and set font properties

The positioning of titles and their font properties can be modified. The xy positioning is in axis limits of 0 to 1. Over 1 positions titles outside the plot axes.

In [19]:
def add_plot_title(ax, title1, title2):

    ax.annotate(title1, xy=(0, 1.15), xycoords='axes fraction', fontsize=14,
                horizontalalignment='left', verticalalignment='top', fontweight="normal")
    ax.annotate(title2, xy=(0, 1.07), xycoords='axes fraction', fontsize=18,
                horizontalalignment='left', verticalalignment='top', fontweight = 'bold')

### Function to add an inset label

The function 'add_inset_label' sets part of text that will be displayed inset inside the plot axes, and the rest of the text is variable through the function arguments. The positioning and font properties of the inset text is also set here.

In [20]:
def add_inset_label(ax, label_start, today):
    label_text = f"{label_start} ending {today}"
    ax.annotate(label_text, xy=(0.03, 0.9), xycoords='axes fraction', fontsize=12,
                horizontalalignment='left', verticalalignment='top', fontweight="normal")

In [21]:
def add_inset_label_mko(ax, label_text):
    blue = '#1e47b0'
    ax.annotate(label_text, xy=(0.03, 0.8), xycoords='axes fraction', fontsize=12,
                horizontalalignment='left', verticalalignment='top', fontweight="bold", color=blue)

### Function to add the SIO logo

This function adds the SIO logo to the graphic. The logo is a svg file which needs to be converted into a png format in order to be plotted. Give it a high enough resolution to not appear pixelated in the final graphic. Once the logo is added to the plot, the intermediate png file can be deleted.

In [22]:
def add_sio_logo(fig, logo_file, xpos, ypos):

    # Convert the logo svg file to a png file with the
    # given scale and dpi
    logo_png = 'logo.png'
    svg2png(url=logo_file, write_to=logo_png, scale=10, dpi=300)

    logo = mpimg.imread(logo_png)

    fig.add_axes([xpos, ypos, 0.2, 0.2], anchor='SE', zorder=1)

    plt.imshow(logo)
    plt.axis('off')

    # Delete the logo png version
    os.remove(logo_png)


### Function to set the image size and save the graphic

The function 'save_graphic' will set the size and resolution of the final graphic. A pdf version and a png version are saved here.

In [23]:
def save_graphic(fig, pdf_file, png_file):

    fig_width, fig_height = fig.get_size_inches()

    # For pdf
    width_in = 10
    height_in = (fig_height/fig_width) * width_in

    fig.set_size_inches(width_in, height_in)

    # Save to a high dpi so that logo png file has a high resolution
    fig.savefig(pdf_file, facecolor='w', edgecolor='w',
                orientation='landscape', format=None,
                transparent=False, bbox_inches='tight', dpi=600)

    # For png
    # Want a png of height 500px

    # First save figure as a png and then scale to size needed

    png_dpi = 300

    fig.savefig(png_file, facecolor='w', edgecolor='w',
                orientation='landscape', dpi=png_dpi, bbox_inches='tight')

    # Now resize the image to be width 1000px for the
    # keelingcurve website
    img = Image.open(png_file)
    img_width, img_height = img.size

    # width_px = 1000
    # img_scale = (width_px / img_width)
    # height_px = int((float(img_height) * float(img_scale)))

    height_px = 500
    img_scale = (height_px / img_height)
    width_px = int((float(img_width) * float(img_scale)))

    img = img.resize((width_px, height_px), Image.LANCZOS)
    img.save(png_file)


### Function to create a Context Manager

This allows a message to display text while a function is running and it is usefull for indicating that function is running and nothing has stalled.

In [24]:
class ShowProgressContextManagerWidgets:

    def __init__(self, out, command=''):
        self.out = out
        self.command = command
        
    def __enter__(self):
        css_style = "<style>span.start { margin:0; padding: 0; color: blue; }</style>"
        html_str = f'<span class="start">Running {self.command}...</span>'

        self.out.clear_output()
        
        with self.out:
            display(HTML(css_style))
            display(HTML(html_str))

        
    def __exit__(self, exc_type, exc_value, exc_tb):
        
        css_style = "<style>span.end { margin: 0; padding: 0; color: green; }</style>"
        html_str = '<span class="end">Finished</span>'

        self.out.clear_output()

        with self.out:
            display(HTML(css_style)) 
            display(HTML(html_str))


# **Steps to create the graphic**

### Change any properties in the section [Functions to modify the graphic](#functions-to-modify-graphic)

### Comment out any function in the overall function [create_graphic](#function-to-create-the-graphic) to remove an element from the final graphic

Comment out a line by adding a # symbol in front of a statement


## **Set properties to use for matplotlib (plotting engine)**

In [25]:
set_matplotlib_properties()

## **Set plot limits and labeling**

### Set titles and axes labels

In [26]:
xlabel = ''
ylabel = "$\mathregular{CO}\\bf{_2}$" + " Concentration (ppm)"

title1 = ""
title2 = "Carbon dioxide concentration at Mauna Loa Observatory*"

### Set yaxis min and max limits

In [27]:
ymin = 310
ymax = 430

### Set xaxis min and max limits

In [28]:
xmin = min(date_co2)

# Get current date as decimal year
now = datetime.now()
xmax = dt2t(now)

<a name="function-to-create-the-graphic"></a>
## **Function to create the graphic**

In [29]:
def create_graphic(msg_out, graphic_out, xmin, xmax, ymin, ymax, xlabel, ylabel, title1, title2, date_co2, co2, btn):

    msg_out.clear_output()
    graphic_out.clear_output()

    # create a figure container
    fig = plt.figure(figsize=(10, 7))

    # Set axes
    # Add padding for exterior plot text
    ax = plt.axes([0.11, 0.12, 0.83, 0.54])

    # ---------------------------------------------------
    # Context manager 
    # 
    # Used to show messages while creating the animiation
    # ---------------------------------------------------

    with ShowProgressContextManagerWidgets(msg_out, 'Create Graphic') as progress:

        # ------------------------
        # Add gradient under curve
        # ------------------------

        # For gradient max, use last date of data rather than current date
        xmax_gradient = max(date_co2)

        area_color = '#73a7e6'

        gradient_fill(date_co2, co2, fill_color=area_color, xmin=xmin, xmax=xmax_gradient,
                    ymin=ymin, ymax=ymax, ax=ax, alpha_bottom=0.1, alpha=1.0)

        # -------------
        # Plot the data
        # -------------

        # Set plotting zorder above the gradient so the plot points are on top

        ax.plot(date_co2, co2, 'o', color='black',markersize=3.5, 
                markerfacecolor='black', markeredgecolor='black', markeredgewidth=0, zorder=5)

        # Add a gap in the line when it skips a date
        # if between, mko_decimal_start_date and mko_decimal_end_date, don't plot fit line
        df = pd.DataFrame({'date_fit':date_co2_fit, 'co2_fit':co2_fit})
        df_left = df[df['date_fit'] < mko_decimal_start_date]
        df_right = df[df['date_fit'] > mko_decimal_end_date]

        ax.plot(df_left['date_fit'], df_left['co2_fit'], '-', color='black', linewidth=0.5, zorder=4)
        ax.plot(df_right['date_fit'], df_right['co2_fit'], '-', color='black', linewidth=0.5, zorder=4)


        # differentiate MKO data
        blue = '#1e47b0'
        ax.plot(date_co2_mko, co2_mko, 'o', color=blue,markersize=3.5, 
                markerfacecolor=blue, markeredgecolor=blue, markeredgewidth=0, zorder=7)            

        ax.plot(date_co2_fit_mko, co2_fit_mko, '-', color=blue, linewidth=0.5, zorder=6)


        # ------------------------------------------------
        # Get todays date in various formats for labeling
        # ------------------------------------------------
        todays_date_moyr, todays_date_modyyr, todays_decimal, todays_year = get_todays_date_variations()

        # ---------------
        # Add plot titles
        # ---------------
        add_plot_title(ax, title1, title2)

        # --------------
        # Add inset text
        # --------------
        add_inset_label(ax, 'Full record', todays_date_modyyr)

        add_inset_label_mko(ax, "*Maunakea data in blue")

        # ------------------------
        # Add SIO logo to the plot
        # ------------------------
        # set position of the logo (units are 0 to 1 of the plot area)
        xpos = 0.715
        ypos = 0.155

        add_sio_logo(fig, logo_file, xpos, ypos)

        # ----------------------------------------
        # Set plot properties

        # Set after the other figure properties to
        # update the default properties
        # -----------------------------------------
        set_plot_props(ax, fig, xmin, xmax, ymin, ymax, xlabel, ylabel)

        # ------------
        # Save graphic
        # ------------

        fig = plt.gcf()

        save_graphic(fig, pdf_file, png_file)

    with graphic_out:

        plt.show()

<a name='button-to-create-and-save-the-graphic'></a>
## **Button to create and save the graphic**

In [30]:
msg_out = widgets.Output()
graphic_out = widgets.Output()

button = widgets.Button(description='Create Graphic', button_style='primary')
button.on_click(functools.partial(create_graphic, msg_out, graphic_out, xmin, xmax, ymin, ymax, xlabel, ylabel, title1, title2, date_co2, co2))
display(button)

display(msg_out)
display(graphic_out)

Button(button_style='primary', description='Create Graphic', style=ButtonStyle())

Output()

Output()

## **Button to download PDF & PNG files of the graphic**

The download button only works when the notebook is run with Google Colab

If the notebook is run on a local machine, the graphics are inside the plots directory

In [31]:
def download_files(msg_out, btn):

    msg_out.clear_output()

    try:
        files.download(png_file)
        files.download(pdf_file)
    except:
        with msg_out:
            print("If running notebook in Google Colab:")
            print("Need to create the graphic first.")
            print(f"\nIf running in a python environment, the graphics are in the plots directory.")


msg_out = widgets.Output()

button = widgets.Button(description="Download Graphic Files", button_style="primary")

layout = widgets.Layout(width='auto', height='auto') #set width and height

button = widgets.Button(
    description='Download Graphic Files',
    disabled=False,
    display='flex',
    flex_flow='column',
    align_items='stretch', 
    button_style='primary',
    layout = layout
)   

button.on_click(functools.partial(download_files, msg_out))

display(button)
display(msg_out)

Button(button_style='primary', description='Download Graphic Files', layout=Layout(height='auto', width='auto'…

Output()